In [1]:
!pip install selenium
#!pip install webdriver-manager
!apt-get update 
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.n

In [2]:
import requests
from bs4 import BeautifulSoup
import html
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [3]:
def scrape_subtopics(soup):
  topics = []
  links = []
  subtopics = []
  content = soup.find_all("a", {'class' : "lesson-navigation-link second-level-link"})
  for item in content:
    #get subtopic text
    subtopic = item.text
    subtopics.append(subtopic)
    #get parent topic
    parent = item.parent.parent.parent.find('h6').text
    topics.append(parent)
    #get subtopic link
    link = item['href']
    links.append(link)
  return topics, subtopics, links

In [4]:
url = "https://daryn.online/course/511-algebra-dgane-analyz-bastamalary-10-synyp"
driver.get(url)
soup = BeautifulSoup(driver.page_source,features="html.parser")

topics, subtopics, links = scrape_subtopics(soup)

In [9]:
df = pd.concat([pd.Series(topics,name='Topic'),pd.Series(subtopics,name='Subtopic'),pd.Series(links,name='Link')], axis=1)
df.head()

,Topic,Subtopic,Link
0,9 сыныптың алгебра курсын қайталау,9 сыныптың алгебра курсын қайталау. Екі айныма...,https://n.daryn.online/lesson/16726-9-synyptyn...
1,9 сыныптың алгебра курсын қайталау,9 сыныптың алгебра курсын қайталау. Екі айныма...,https://n.daryn.online/lesson/16727-9-synyptyn...
2,9 сыныптың алгебра курсын қайталау,9 сыныптың алгебра курсын қайталау. Тізбектер....,https://n.daryn.online/lesson/16728-9-synyptyn...
3,9 сыныптың алгебра курсын қайталау,9 сыныптың алгебра курсын қайталау. Тригономет...,https://n.daryn.online/lesson/16729-9-synyptyn...
4,"Функция, оның қасиеттері және графигі",Функция және оның берілу тәсілдері. Функциялар...,https://n.daryn.online/lesson/16758-funktsyya-...


In [6]:
df.to_excel("output.xlsx")

In [11]:
#merge cells for a better format 
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook

wb = Workbook()
ws = wb.active
for r in dataframe_to_rows(df, index=True, header=True):
    ws.append(r)
#wb = load_workbook(filename='output.xlsx')
#ws = wb.active
ws.delete_cols(1)
ws.delete_rows(2)
first_idx = [] 
last_idx = []
for row in ws.iter_cols(min_col=1, max_col=1, values_only=True):
  unique = set(row)
  for topic in unique:
    first_idx.append(row.index(topic) + 1)
    last_idx.append(len(row) - list(row[::-1]).index(topic))
for i in range(len(first_idx)):
  ws.merge_cells('A'+str(first_idx[i])+':'+'A'+str(last_idx[i]))
wb.save("out_merged.xlsx")